In [1]:
import os, copy, csv
import numpy as np
import pandas as pd
from skimage.transform import resize
import matplotlib.pyplot as plt
from PIL import Image
import PIL
import torch, torchvision
from deeplabv3 import FreiburgTestDataset, FreiburgDepthTestDataset, cf, dn, nut, c2e

## Load validation dataset

In [2]:
batch_size=50

In [3]:
test_orig_img_shape = (480,640)#(768, 1024)

In [4]:
fd_val = FreiburgTestDataset("data/freiburg/val")

## Obtain depth prediction with monodepth2

In [5]:
#change transform to match monodepth2's required size
fd_val.transforms = torchvision.transforms.Compose([Image.fromarray,
                                                     lambda x: x.resize((640, 192), Image.LANCZOS), 
                                                     torchvision.transforms.ToTensor()])
test_dataloader2 = torch.utils.data.DataLoader(fd_val, batch_size=batch_size, shuffle=False)
test_img2 = next(iter(test_dataloader2))

In [6]:
from monodepth2 import estimate_depthmap

In [7]:
test_depth_pred = estimate_depthmap(test_img2)

-> Loading model from  models\mono_640x192
   Loading pretrained encoder
   Loading pretrained decoder


c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
test_img2 = dn(test_img2)

fig,axs=plt.subplots(batch_size,2, figsize=(20,20))
for i in range(2):
    axs[i,0].imshow(c2e(test_img2[i]))
    axs[i,1].imshow(test_depth_pred[i], cmap='gray' )

# Retrieve ground truths

In [9]:
fd_depth_test = FreiburgDepthTestDataset("data/freiburg/val")

In [10]:
test_depth_true = np.zeros((len(fd_depth_test), 192, 640))
for i in range(len(fd_depth_test)):
    test_depth_true[i] = fd_depth_test[i]

# Compare

In [11]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [15]:
a = np.clip(1/test_depth_pred, 0.0, 1.0)
b = test_depth_true[:50]

In [16]:
MAPEs = []

In [17]:
for i in range(a.shape[0]):
    MAPEs.append(mean_absolute_percentage_error(b[i].flatten(), a[i].flatten()))

In [21]:
np.mean(MAPEs)

0.30508909889900093

In [57]:
mean_squared_error(b, a, squared=False)

0.18173999869727103